# ChatBot

## Imagen general / problema a resolver

Este modelo debe clasificar cada conversacion y comprobar si es de un contexto u otro

El objetivo de este modelo es crear un chatbot que responda consultas de los usuarios

Vamos a tener los siguientes datos sobre los emails

* Mensaje
* Contexto

Con la informacion de que disponemos, se trata de un sistema **supervisado**

Estamos ante un problema tipico de **clasificación**, ya que queremos separar los mensajes de un contexto de otro


## Obtención de datos

Los datos los crearemos nosotros

Importamos Numpy, Pandas, Matplotlib, Seaborn

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Creamos los datos para nuestro entrenamiento

In [40]:
message_data = pd.read_csv("/content/drive/MyDrive/Message_Dataset/message_data.csv")
message_data

,Unnamed: 0,message,context
0,0,¿Cuál es el precio de un coche con 200 caballos?,1
1,1,¿Cuánto cuesta un coche de 2019?,1
2,2,"Quiero un coche que tenga más de 300 caballos,...",1
3,3,¿Hay coches de la marca Toyota disponibles?,1
4,4,¿Cuánto cuesta un coche con caja de cambios au...,1
...,...,...,...
352,352,Estoy buscando un modelo de Jeep con menos de ...,1
353,353,¿Hay algún Land Rover disponible con sistema d...,1
354,354,Necesito un Honda con caja de cambios automáti...,1
355,355,Quiero un Subaru del año 2017 o posterior con ...,1


## Preparación de los datos

### Randomizado de los datos

Necesitamos randomizar la posicion de los datos debido a que todos los de contexto 0 estan al final y los de contexto 1 al inicio

In [41]:
message_data = message_data.sample(frac=1)
message_data

,Unnamed: 0,message,context
142,142,¿Qué planes tienes para las vacaciones de verano?,0
221,221,¿Hay algún modelo con más de 7 marchas?,1
249,249,Estoy buscando algo con año 2018 o más nuevo.,1
129,129,¿Has visto la película que estrenaron ayer?,0
40,40,¿Cuál es el precio promedio de un coche de la ...,1
...,...,...,...
7,7,¿Cuántas marchas tienen los coches de la marca...,1
263,263,"Quiero un Ferrari rojo, preferiblemente con me...",1
175,175,¿Tienes alguna mascota?,0
293,293,Estoy interesado en algo de la marca Maserati ...,1


### Creacion de la funcion tokenizadora
Necesitamos crear una funcion tokenizadora para separar las palabras del texto y asi poder hacer una mejor vectorización

In [42]:
import string

punctuation = set(string.punctuation)
def tokenize(sentence):
  tokens = []
  for token in sentence.split():
    new_token =[]
    for character in token: 
      if character not in punctuation:
        new_token.append(character.lower())
    if new_token:
      tokens.append("".join(new_token))
  return tokens

Separación de la matrix de características  X  de los datos de salida  y  (el target)

In [43]:
X = message_data["message"]
X

142    ¿Qué planes tienes para las vacaciones de verano?
221              ¿Hay algún modelo con más de 7 marchas?
249        Estoy buscando algo con año 2018 o más nuevo.
129          ¿Has visto la película que estrenaron ayer?
40     ¿Cuál es el precio promedio de un coche de la ...
                             ...                        
7      ¿Cuántas marchas tienen los coches de la marca...
263    Quiero un Ferrari rojo, preferiblemente con me...
175                              ¿Tienes alguna mascota?
293    Estoy interesado en algo de la marca Maserati ...
224    Estoy buscando un coche con cambio automático,...
Name: message, Length: 357, dtype: object

In [44]:
y = message_data["context"]
y

142    0
221    1
249    1
129    0
40     1
      ..
7      1
263    1
175    0
293    1
224    1
Name: context, Length: 357, dtype: int64

## Creacion de los datos de prueba

Separamos el dataset en dos partes

*    Datos de entrenamiento (training set) - 80%
*    Datos de prueba (test set) - 20%

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=11001100)

## Conversion texto a vector numerico

Deberemos convertir el texto a vector numerico y ver la relevancia que tiene cada palabra en un mensaje y si influye en el resultado de ser 0 o 1

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(tokenizer = tokenize, binary = True)
X_train_tfidf = count_vect.fit_transform(X_train)

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## Entrenamiento del modelo

### Modelo MultinomialNB

In [47]:
from sklearn.naive_bayes import MultinomialNB

message_model = MultinomialNB().fit(X_train_tfidf, y_train)

### Aciertos y Fallos

In [48]:
y_pred = message_model.predict(count_vect.transform(X_test))

print("Aciertos")
aciertos = np.count_nonzero(y_test == y_pred)
print(f"Hubo {aciertos} Aciertos y {len(y_test) - aciertos} fallos")
print(f"Hubo un porcentaje de {aciertos * 100 / len(y_test)}% aciertos")

Aciertos
Hubo 35 Aciertos y 1 fallos
Hubo un porcentaje de 97.22222222222223% aciertos


In [49]:
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""quiero ver unos coches too chulos"""]))) # la grafica mostraria coches randoms xd
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Me gustaria ver coches de la marca BMW y con 500000km"""])))
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Enseña los coches de 100CC y con cambio de marcha manual"""])))
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Enseñame algo que sea un mercedes de 2010"""])))
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Me gustaria ver algo que sea del año 2020"""])))
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Me gustaria ver algo que sea del que tenga 200cc de cilindrada"""])))
# Mensaje random
print(message_model.predict(count_vect.transform(["""El final de snk es kk"""])))
# Mensaje random
print(message_model.predict(count_vect.transform(["""Quiero una hamburguesa del año 2000"""])))
# Mensaje random
print(message_model.predict(count_vect.transform(["""Pagame perro >:V"""])))
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Muestrame los coches de madrid"""])))

[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]


### Modelo LinealSVC

In [50]:
from sklearn.svm import LinearSVC

message_model = LinearSVC().fit(X_train_tfidf, y_train)

### Aciertos y Fallos

In [51]:
y_pred = message_model.predict(count_vect.transform(X_test))

print("Aciertos")
aciertos = np.count_nonzero(y_test == y_pred)
print(f"Hubo {aciertos} Aciertos y {len(y_test) - aciertos} fallos")
print(f"Hubo un porcentaje de {aciertos * 100 / len(y_test)}% aciertos")

Aciertos
Hubo 35 Aciertos y 1 fallos
Hubo un porcentaje de 97.22222222222223% aciertos


In [52]:
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""quiero ver unos coches too chulos"""]))) # la grafica mostraria coches randoms xd
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Me gustaria ver coches de la marca BMW y con 500000km"""])))
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Enseña los coches de 100CC y con cambio de marcha manual"""])))
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Enseñame algo que sea un mercedes de 2010"""])))
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Me gustaria ver algo que sea del año 2020"""])))
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Me gustaria ver algo que sea del que tenga 200cc de cilindrada"""])))
# Mensaje random
print(message_model.predict(count_vect.transform(["""El final de snk es kk"""])))
# Mensaje random
print(message_model.predict(count_vect.transform(["""Quiero una hamburguesa del año 2000"""])))
# Mensaje random
print(message_model.predict(count_vect.transform(["""Pagame perro >:V"""])))
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Muestrame los coches de madrid"""])))

[1]
[1]
[1]
[1]
[1]
[1]
[0]
[0]
[0]
[1]


### Modelo Random Forest

In [61]:
from sklearn.ensemble import RandomForestClassifier

message_model = RandomForestClassifier(max_depth=20).fit(X_train_tfidf, y_train)

### Aciertos y fallos

In [62]:
y_pred = message_model.predict(count_vect.transform(X_test))

print("Aciertos")
aciertos = np.count_nonzero(y_test == y_pred)
print(f"Hubo {aciertos} Aciertos y {len(y_test) - aciertos} fallos")
print(f"Hubo un porcentaje de {aciertos * 100 / len(y_test)}% aciertos")

Aciertos
Hubo 35 Aciertos y 1 fallos
Hubo un porcentaje de 97.22222222222223% aciertos


In [63]:
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""quiero ver unos coches too chulos"""]))) # la grafica mostraria coches randoms xd
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Me gustaria ver coches de la marca BMW y con 500000km"""])))
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Enseña los coches de 100CC y con cambio de marcha manual"""])))
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Enseñame algo que sea un mercedes de 2010"""])))
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Me gustaria ver algo que sea del que tenga 200cc de cilindrada"""])))
# Mensaje random
print(message_model.predict(count_vect.transform(["""El final de snk es kk"""])))
# Mensaje random
print(message_model.predict(count_vect.transform(["""Quiero una hamburguesa del año 2000"""])))
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Quiero coches de 2004"""])))
# Mensaje random
print(message_model.predict(count_vect.transform(["""Pagame perro >:V"""])))
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Muestrame los coches de madrid"""])))
# Mensaje de consulta
print(message_model.predict(count_vect.transform(["""Muestrame los bmw de madrid del año 2018"""])))

[1]
[1]
[1]
[1]
[1]
[0]
[0]
[1]
[0]
[1]
[1]


## Exportación del modelo

In [56]:
import joblib

In [57]:
joblib.dump(message_model, "message_model.pkl")

['message_model.pkl']